In [93]:
import requests
import pandas as pd
import plotly.express as px
import json
from json import dumps
import datetime
import os

QUE FAIRE A PARIS

In [25]:
url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/que-faire-a-paris-/records?limit=100"

#headers = {'apiKey': os.environ.get('RATP')}
p = {}

response = requests.get(url, params=p)

In [26]:
response.content

b'{"total_count": 2543, "results": [{"id": "11114", "url": "https://www.paris.fr/evenements/paris-11e-ou-donner-vos-vetements-a-paris-11114", "title": "Paris 11e\\u00a0: o\\u00f9 donner vos v\\u00eatements \\u00e0 Paris", "lead_text": "Pulls, blousons, pantalons\\u2026 Ne laissez plus vos vieux v\\u00eatements s\'entasser dans les armoires, et ne les jetez pas non plus\\u00a0! En donnant \\u00e0 ceux qui en ont besoin vous ferez une bonne action et vous donnerez une seconde vie \\u00e0 vos v\\u00eatements.", "description": "<p>la <a href=\\"https://onzemillepotes.com/\\">Bagagerie Onze Mille Potes</a> a des besoins de v\\u00eatements chauds et de produits d\'hygi\\u00e8ne. <br></br></p><p>Les besoins de v\\u00eatements\\u00a0: tee-shirts, \\u00e9charpes, bonnets, gants, chaussettes, sous-v\\u00eatements neufs, duvets\\u2026</p><p>Les besoins de produits d\'hygi\\u00e8ne\\u00a0: gel douche, shampoing, brosse \\u00e0 dent, dentifrice\\u2026 <br></br></p><p>Les v\\u00eatements doivent \\u

In [38]:
liste_events = response.json().get("results")

VELIB A PARIS

PRENDRE LES VELIBS LE MATIN ET LE SOIR (EVTL A MIDI ETC.) POUR VOIR LES FLUX SUR LA JOURNEE

L'API DE PARIS AUTORISE UNIQUEMENT 100 STATIONS, ON UTILISE DONC l'API VELIB POUR AVOIR TOUTES LES STATIONS MAIS ON N'A PAS LES COORDONNEES ET L'ADRESSE ETC DONC IL FAUT FAIRE UN MAPPING AVEC LE CSV DE PARIS (VIA STATION CODE)

In [231]:
url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/velib-disponibilite-en-temps-reel/records?limit=100"

url = "https://velib-metropole-opendata.smovengo.cloud/opendata/Velib_Metropole/station_status.json"

#headers = {'apiKey': os.environ.get('RATP')}
p = {}

response = requests.get(url, params=p)

In [252]:
df_coordonnees_stations = pd.read_csv('src/velib-emplacement-des-stations.csv', sep=";")

In [253]:
df_coordonnees_stations['lat'] = df_coordonnees_stations["Coordonnées géographiques"].apply(lambda x : x.split(',')[0]).astype('float64')
df_coordonnees_stations['lon'] = df_coordonnees_stations["Coordonnées géographiques"].apply(lambda x : x.split(',')[1]).astype('float64')

In [254]:
df_coordonnees_stations=df_coordonnees_stations.rename(columns={"Identifiant station":'stationCode'})

ON S'OCCUPE DES INFOS DE L'API

In [255]:
liste_dispos_velib = response.json().get("data").get("stations")

In [256]:
df_dispos_velib = pd.DataFrame(liste_dispos_velib)
df_dispos_velib.head()


,station_id,num_bikes_available,numBikesAvailable,num_bikes_available_types,num_docks_available,numDocksAvailable,is_installed,is_returning,is_renting,last_reported,stationCode
0,213688169,16,16,"[{'mechanical': 13}, {'ebike': 3}]",18,18,1,1,1,1733145301,16107
1,17278902806,7,7,"[{'mechanical': 1}, {'ebike': 6}]",13,13,1,1,1,1733145280,44015
2,36255,3,3,"[{'mechanical': 0}, {'ebike': 3}]",17,17,1,1,1,1733145245,9020
3,251039991,11,11,"[{'mechanical': 2}, {'ebike': 9}]",11,11,1,1,1,1733144898,14111
4,85002689,4,4,"[{'mechanical': 2}, {'ebike': 2}]",56,56,1,1,1,1733145315,14014


In [257]:
df_dispos_velib['stationCode']=df_dispos_velib['stationCode'].astype('int64')

ON FAIT UN JOIN DES DEUX TABLES POUR AVOIR LES DISPOS ET TOUTES LES COORDONNEES 

In [258]:
inner_join = pd.merge(df_dispos_velib,  
                      df_coordonnees_stations,  
                      on ='stationCode',  
                      how ='right') 

AFFICHAGE DES STATIONS AVEC LE NOMBRE DE VELOS DISPOS

In [260]:

fig = px.scatter_mapbox(inner_join,lat="lat", lon="lon",  hover_name="Nom de la station", height=700, size='numBikesAvailable',
mapbox_style="open-street-map",title='Carte des plaques',zoom=10,color_continuous_scale=["red","orange","yellowgreen","green"])
fig.update_layout(title_x=0.5)
fig.show()

In [261]:
inner_join["datetime_update"]=datetime.datetime.now()

In [263]:
filename_data = 'velibs.csv'

if filename_data in os.listdir("csv//"):
    print("le fichier existe")
    on_a_deja_les_datas_du_jour = True
    inner_join = pd.DataFrame(pd.read_csv('csv/'+filename_data), dtype='str')
else:
    on_a_deja_les_datas_du_jour = False
    print("Le fichier existe pas")

Le fichier existe pas


In [264]:
if (on_a_deja_les_datas_du_jour==False):
    inner_join.to_csv(f"csv/velibs.csv", index=False)